# Final Project Notebook: Halicioglu Data Science Institute Launch Submission

## Group Members

Dang Le: A14269781

Stephan Somers: A12349848

Elena Dreisbach: A13169440

Tarun Pasumarthi: A12098060

## Introduction and Background

It is a known fact that veterans are at a higher risk of suicide in comparison to the general population, however the extreme degree to which this is true is not nearly as well understood. According to the U.S. Department of Veteran Affairs, in comparison to the general population deployed veterans are at a 41% higher risk of suicide and non-deployed veterans are at a 61% higher risk. These abnormally high rates of suicide could be attributed to many factors that come with the high stress risk of joining the military. Research has been done showing an inverse relation between suicide rates and socioeconomic status when analyzed at a community level, and we are interested how this might relate to the statistic for veterans (Rehkopf DH, Buka SL 2006).

[NEEDS TO BE EDITED: We hypothesize that lower socioeconomic status will increase suicide rates, while better socioeconomic well-being will lower suicide rates among veterans. We believe that once veterans leave the military, they lose the structure in their life which they are used to after being placed in high-stress situations. This removal of constant structure might place veterans in a lower than average socioeconomic status, thus placing them at higher risks for mental health issues related to suicide.] 

This is incredibly important as this could help address where veterans suicide resources can best be utilized, as well as identifying which groups need more support post-service.

## Data Description

The following dataset shows the comparison of state suicide rates between American civilians and veterans - we will use this to compare veteran suicide to socioeconomic status.

Dataset Name: Suicide Rates by State

Link to the dataset: https://backhome.news21.com/interactive/suicide-interactive/

Number of observations: 249,874

<br>

The following dataset shows the median household income of each U.S. state. We will use this and the next two datasets to create a visualization of socioeconomic status.

Dataset Name: Median Household Income by State

Link to the dataset: https://www.census.gov/data/tables/time-series/demo/income-poverty/historical-income-households.html

Number of observations: 1,768

<br>

The following dataset shows the number of employed versus unemployed Americans, and is being used to find unemployment rates within each state. We will use this and the next dataset to create a visualization of socioeconomic status.

Dataset Name: Labor Force Data By County

Link to the dataset: https://www.bls.gov/lau/

Number of observations: 155,826,564*

<br>

The following dataset lists the housing price index by state; we ultimately decided not to use this dataset due to the convoluted nature of the housing price index itself. The process would have been incredibly time-consuming to coordinate this dataset with the rest of our analyses.

Dataset Name: Housing Price Index by State

Link to the dataset: https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx

Number of observations: 1,632

<br>

We plan to look at the datasets to discover if there is a correlation between socioeconomic status and veteran suicide rate. We are using median household income and unemployment rates to judge the socioeconomic status of any particular state.

## Data Cleaning/Pre-Processing

The first dataset that we cleaned was the "Suicide Rates by State". For this, it was necessary to ____


The second dataset was that of "Labor Force Data". Here, the data was broken down by county - as such, we aggregated the rows that shared the same state into a new 'df_unemployment_by_state' dataframe in order to work with it more easily.

The third dataset was "Median Household Income by State". To clean this data, the first task required a decision to choose data which was either normalized to the current dollar or to the 2016 dollar. As our data is older, it made more sense to go with the 2016 dollar normalization. Right off the bat, the data did not import properly into the Jupyter Notebook due to the formatting of the Excel file. In the excel spreadsheet, each column was labeled by two rows. We fixed this in our Jupyter notebook by looking at both rows and setting our columns to the concatenation of these two rows. There were a few rows that contained labels or were otherwise useless, we dropped these. We then had a dataset which had each state alongside standard error and median income from 1984 to 2016. Standard error was not required for our data analysis so it was also dropped. There were a few duplicate year columns - it appeared that there may have been some controversy over which was more accurate, so both were left in the dataset. 

## Data Visualization

## Data Analysis and Results

## Privacy/Ethics Considerations

The data we are accessing is publicly available for use, and we were unable to find any permission restrictions. However, as the data does concern veterans there is an increased necessity for protection of data which involves identifying information. We were lucky enough to find a dataset which had minimal identifying information (only involving state, year of death, and number of suicides within that state), and we dropped potentially identifying columns like gender and age group. One potential issue arises in states where there are few suicides within a given year - it would not be incredibly difficult to track down certain individuals within the armed forces who had committed suicide in a specific state during a specific year. The one potential ethical issue that we have discussed involves veterans data, however our datasets do not include any protected information as they are representative of the overall population rather than specific people. If something of this sort were to arise, we could easily mitigate the issue by removing the sensitive information (such as names or zip codes).

There is always the potential for bias within datasets; however, as our datasets are quite large, the potential for bias is significantly reduced. Additionally, the majority of our data was collected by government agencies (such as the U.S. Census Bureau and The Center for Disease Control) who are using reliable data collection methods so as to limit the amount of year-to-year bias within the data collection process. However, as our data was pulled from a census survey, there exists some inherent bias within the survey process. For example, DJ Patil mentioned in his lecture that felons are often not counted within the US Census. Additional biases such as these could easily exist within the dataset. There is honestly not much that we can do about this bias in regards to our data analysis, but it is certainly important to keep in mind as we move forward with this project. If we identify any additional issues in regards to privacy or bias, we will do our best remove that problematic data (in cases of privacy breaches or similar incidences). With datasets as large as ours, removal of a relatively small amount of data should not have a strong impact on our data analysis.

## Conclusions and Discussion